# Motivation

- What is your dataset?

- Why did you choose this/these particular dataset(s)?

- What was your goal for the end user's experience?

The aim of this project is to explore the social science community. To construct this "community" of social scientists, we found inspiration by looking at department for Social Science at Copenhagen University. The Social Science department have five different sciences: Sociology, psychology, economy, anthropology, and political science. With these five categories we found a list of scientists for each category on wikipedia and from these five lists we gathered the wikipedia page for each scientist, which then is just necessary to clean and preprocess to make possible to use for network analysis and NLP models.  

By looking into these disciplines’ social scientists we hope to find patterns in the connections both within the five categories but also between the five social sciences. We will first seek to answer the question of, what characteristics does the community of the five social sciences have? We will examine this question by looking into the over network properties of each followed by a network analysis. Thereafter we will examine the communities of the social sciences, by doing this we will answer these questions: Are the traditional categories within the social science community actually also apt/suitable for the social science community? Or are the categorizations just a historic relic which we should reject, and then embrace some other maybe more relevant categorizations within the social sciences? By looking into this it becomes possible to either deny or affirm the different scientists relevance within the different disciplines of the social science community. 

No clue what our goal for the end users experience is? 

# Basic

- Write about your choices in data cleaning and preprocessing

- Write a short section that discusses the dataset stats (here you can recycle the work you did for Project Assignment A)

# Tools, theory and analysis. Describe the process of theory to insight

- Talk about how you've worked with text, including regular expressions, unicode, etc.
Describe which network science tools and data analysis strategies you've used, how those network science measures work, and why the tools you've chosen are right for the problem you're solving.

- How did you use the tools to understand your dataset?

#### ANALYSE STRUKTUR: 
Explain the overall idea

- Analysis step 1
    - explain what you're interested in
    - explain the tool
    - apply the tool
    - discuss the outcome

- Analysis step 2
    - explain what you're interested in
    - explain the tool
    - apply the tool
    - discuss the outcome

- Analysis step 3,
... and so on until the analysis is done

# Discussion. Think critically about your creation

- What went well?,
- What is still missing? What could be improved?, Why?

# Constributions

- You should write (just briefly) which group member was the main responsible for which elements of the assignment. (I want you guys to understand every part of the assignment, but usually there is someone who took lead role on certain portions of the work. That’s what you should explain).

- It is not OK simply to write "All group members contributed equally".